<a href="https://colab.research.google.com/github/Tariquzzaman-faisal/VITD/blob/main/bilstm_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setting up project

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd "/content/drive/MyDrive/Research/Shared Task/Violence Inciting Text Detection (VITD) Bangla"

/content/drive/MyDrive/Research/Shared Task/Violence Inciting Text Detection (VITD) Bangla


In [ ]:
%pwd

'/content/drive/MyDrive/Research/Shared Task/Violence Inciting Text Detection (VITD) Bangla'

In [ ]:
%ls

dataset/  embedding/  models/  notebooks/


# Loading dataset

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df_train = pd.read_csv('dataset/task/train.csv')
df_val = pd.read_csv('dataset/task/dev.csv')

In [ ]:
label_mapping = {
    0: "Non-Violence",
    1: "Passive Violence",
    2: "Direct Violence"
}

# Use the replace function to update the values in the 'label' column
df_train['label'] = df_train['label'].replace(label_mapping)
df_val['label'] = df_val['label'].replace(label_mapping)

In [ ]:
df_train

,text,label
0,যে দেশে সন্ত্রাসরা দেশ চালায়সে দেশে শান্তি কিভ...,Passive Violence
1,এই বিচার শেষ বিচার নয়।আসল বিচার হবে আল্লাহর আদ...,Non-Violence
2,আরব দেশগুলোকে বলব ভারতের সাথে সব ব্যবসা বাণিজ্...,Direct Violence
3,দেশটা সুস্থ নাই,Non-Violence
4,আপনার কথা দুঃখ জনক আগে বিডিও থাকলে কেন ধরা হলন...,Non-Violence
...,...,...
2695,"হল বন্ধ করার আগে নিউমার্কেট বন্ধ করা উচিৎ,,।",Direct Violence
2696,এ স্বাধীন দেশে ভোটটা অন্ততঃ আমাদের দিতে দেন।জা...,Non-Violence
2697,আল হামদুলিল্লাহ্...প্রিয় ভিউয়ার্স আপনাদের ভালব...,Non-Violence
2698,এই গুলা ত বিচার করবায় ঐ তোমরা ত ইন্ডিয়ান না,Non-Violence


In [ ]:
df_val

,text,label
0,পাডা পুতার মাঝখানে পরে সাধারণ ২ মানুষের জিবন শ...,Non-Violence
1,করোনার চাপে অনেক কিছু বন্ধ ও অনেক বিধি নিষেধ ক...,Non-Violence
2,সঠিক তদন্ত করতে হবে। বিচারের আওতায় আনতে হবে য...,Non-Violence
3,যে লোকটা মারা গেছে তার কি হবে তার দায়ভার কে ন...,Non-Violence
4,নিউ মার্কেট এবং গুলিস্থান মার্কেটের ব্যবসায়ীর...,Passive Violence
...,...,...
1325,নাটক টা সুন্দর ভাবে সাজিয়েছে আরো কত কিছু দেখত...,Passive Violence
1326,নোংরা দেশ আর নোংরা জাতি হচ্ছে ভারত এঁরা কি বুঝ...,Passive Violence
1327,জে ছেলে মারা গেছে ওর কি হবে,Non-Violence
1328,এরাই নৈরাজ্য সৃষ্টি করছে,Passive Violence


In [ ]:
!pip install tensorflow

In [ ]:
from keras.utils.np_utils import to_categorical
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
# Multi label binarizer skipped because all my data have only 1 label

In [ ]:
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional, \
        GlobalMaxPool1D, Input, Flatten, MaxPooling1D, SpatialDropout1D, Activation

In [ ]:
from keras.callbacks import EarlyStopping
from numpy import array
from sklearn.metrics import classification_report

In [ ]:
!pip install gensim

In [ ]:
import gensim
from gensim import models
from gensim.models import Word2Vec

In [ ]:
!pip install fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.11.1-py3-none-any.whl (227 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4199774 sha256=ee993f40f32cb98121f31e5894f8926b8aa37181e1938c2fb97fb2b695ab5100
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fasttext


In [ ]:
import fasttext.util

In [ ]:
train_x = df_train['text'].to_list()
val_x = df_val['text'].to_list()

In [ ]:
len(train_x), len(val_x)

(2700, 1330)

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import numpy as np

# Step 1: Instantiate the LabelEncoder
label_encoder = LabelEncoder()

# Step 2: Fit and transform the training data
train_y_encoded = label_encoder.fit_transform(df_train['label'])

# Step 3: Transform the test data (no need to fit again)
val_y_encoded = label_encoder.transform(df_val['label'])

# Step 4: Get the number of classes
num_classes = len(label_encoder.classes_)

# Step 5: Convert the encoded labels to one-hot encoded format
train_y_onehot = np.eye(num_classes)[train_y_encoded]
val_y_onehot = np.eye(num_classes)[val_y_encoded]

# Step 6: Get the target names (unique labels) from LabelEncoder
target_names = list(label_encoder.classes_)

In [ ]:
train_x

['যে দেশে সন্ত্রাসরা দেশ চালায়সে দেশে শান্তি কিভাবে আশা করবেন?',
 'এই বিচার শেষ বিচার নয়।আসল বিচার হবে আল্লাহর আদালতে। সেইদিন সকল জালেমের মুখোস উন্মোচন হবে',
 'আরব দেশগুলোকে বলব ভারতের সাথে সব ব্যবসা বাণিজ্য বন্ধ করে দেন যারা হিন্দু ব্যবসায়ী আছে তাদের সাথে সব বন্ধ করে দেয়া উচিত',
 'দেশটা সুস্থ নাই',
 'আপনার কথা দুঃখ জনক আগে বিডিও থাকলে কেন ধরা হলনা হিন্দু দের খতি হোতনা ধন্যবাদ আপনাকে',
 'সত্যি এটা মুসলিম করেছে জেনে হিন্দু সম্প্রদায়ের ভাই বোন দের কাছে দূঃখ ও সহমর্মিতা অনুভব করছি!!',
 'একজন বাবা কতোটা অসহায় হলে এই কথা বলতে পারে আল্লাহ তুমি বিচার করো',
 'আবুজাহেলের বংশধর আওয়ামী লীগ বা হাসিনা লীগ জংগী লীগ ডেংগু লীগ চোর লীগ কুত্তা লীগ',
 'সরকারের সব লোক ভারতের দালাল মনে রাখিছ আল্লাহ ছাড় দেয় কিন্তু ছেড়ে দেয়না',
 'এরা দালাল',
 'হিজাবেই নারীর সৌন্দর্য ফুটে ওঠে',
 'ক্ষুব্ধ শিক্ষার্থীদের হল বন্ধ করার দিদ্ধান্তটি আরোবেশি খারাপ করবে!',
 'আপনারা সত্যি তুলে ধরেন নইলে চ্যালেন বন্ধ করেন',
 'ঈমান কত মজবুত দেখ বাংঙ্গালী নারী। এই শিক্ষার কোন দাম নাই মে শিক্ষা প্রকাশ নষ্ট করে। ইহকাল ও ক্ষতিগ্ৰস্ত হয়।',

In [ ]:
val_x

['পাডা পুতার মাঝখানে পরে সাধারণ ২ মানুষের জিবন শেষ আর এর বিচার কোনো দিন হবে না',
 'করোনার চাপে অনেক কিছু বন্ধ ও অনেক বিধি নিষেধ করেছেন তবে নির্বাচন কিন্তু চলছেই কারন কি',
 'সঠিক তদন্ত করতে হবে। বিচারের আওতায় আনতে হবে যে এই কাজ টা করেছে।',
 'যে লোকটা মারা গেছে তার কি হবে তার দায়ভার কে নিবে ছাত্র নাকি ব্যবসায়ী',
 'নিউ মার্কেট এবং গুলিস্থান মার্কেটের ব্যবসায়ীরা হচ্ছে কুকুরের চেয়েও খারাপহয়তোবা আমরা সাধারন মানুষ গেলে এর প্রতিবাদ করতে পারি নাওরা একটা কাপড়ের দাম চায় 15 গুণ বেশি কিন্তু দাম না বললে ওরা গালাগালি করেতাই ঢাকা কলেজের ছাত্র দের অসংখ্য ধন্যবাদ জানাই ওদেরকে উচিত শিক্ষা দেওয়ার জন্য। এবং সেইসাথে বলবো সাধারন পাবলিক যেন ঢাকা কলেজের ছাত্র দের পাশে থাকে ব্যবসায়ীদের উচিত শিক্ষা দিতে হবে',
 'হাইয়া আলাল জিহাদ হাইয়া আলাল জিহাদ',
 'ছাত্রলিগের বিচার চাই',
 'এতদিন পর ঢাকা কলেজের ছাত্ররা উচিত কাজ করেছে। নিউ মার্কেটের ব্যবসায়ীরা ওরা বিক্রির নামে কাস্টমারদের উপর চালায় রির্যাতন। কাপড় তো দুই নাম্বার। নাই কোন ব্রান্ড। দেখলেই দাম বলতে হবে আর এত দাম চায় যে বসুন্ধরাও ফেল। ওরা মেয়েদের উপর যৌন হয়

In [ ]:
train_y_onehot

array([[0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       ...,
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [ ]:
tokenizer=Tokenizer(oov_token = "<OOV>", split=' ')
tokenizer.fit_on_texts(train_x)

In [ ]:
train_encoded=tokenizer.texts_to_sequences(train_x)
train_padded= pad_sequences(train_encoded, padding='post')

In [ ]:
val_encoded=tokenizer.texts_to_sequences(val_x)
val_padded= pad_sequences(val_encoded, padding='post', maxlen=train_padded.shape[1])

In [ ]:
train_padded.shape

(2700, 94)

In [ ]:
max_length = train_padded.shape[1]
vocabulary_size = len(tokenizer.word_index) + 1
EMBEDDING_DIM = 300

In [ ]:
def embedding_creation(EMBEDDING_DIM, word_vectors):

  vocabulary_size=len(tokenizer.word_index)+1
  word_index=tokenizer.word_index
  embedding_matrix = np.zeros((vocabulary_size, EMBEDDING_DIM))

  for word, i in word_index.items():
      try:
          embedding_vector=word_vectors[word]
          embedding_matrix[i]=embedding_vector
      except KeyError:
          embedding_matrix[i]=np.random.normal(0,np.sqrt(0.25),EMBEDDING_DIM)

  embedding_layer=Embedding(vocabulary_size, EMBEDDING_DIM, weights=[embedding_matrix], trainable=False)

  return embedding_layer

### Loading the fasttext model trained on unsupervised data

In [ ]:
# creating informal_FastText embedding layer (IFT)
wv = fasttext.load_model('embedding/model_bn_300.bin')
IFT = embedding_creation(EMBEDDING_DIM, wv)

In [ ]:
# creating a randomly initialized embedding layer (RE)
RE = Embedding(vocabulary_size, EMBEDDING_DIM,input_length = max_length, trainable=True)

In [ ]:
# defining early stopping; stops training when there is no improvement in val_loss for 3 consecutive ecpoch.
# returns best model with least val_loss
earlystop_callback = EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=3,
    verbose=1,
    mode="min",
    restore_best_weights=True,
)

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
checkpoint_path = "models/my_model_checkpoint.h5"
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    monitor='val_accuracy',
    save_best_only=True,
    save_weights_only=True,
    verbose=1
)

In [ ]:
# Define the model architecture outside the tf.function context
def create_model(embedding_layer):
    model = Sequential([
        embedding_layer,
        Bidirectional(LSTM(100, dropout=0.3, return_sequences=True)),
        GlobalMaxPool1D(),
        Dense(16, activation='relu'),
        Dense(3, activation='sigmoid'), # 3 output classes
    ],
    name="Sentiment_Model")
    return model

In [ ]:
emb_X_name_collection = [ [IFT, 'IFT']]

# Instantiate the Embedding layer (assumed to be emb_X_name[0] in your case)
embedding_layer = emb_X_name_collection[0][0]

In [ ]:
# Create the model using the function defined above
model = create_model(embedding_layer)

In [ ]:
# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
train_padded.shape, train_y_onehot.shape

((2700, 94), (2700, 3))

In [ ]:
# Train the model
history = model.fit(train_padded, train_y_onehot, epochs=100, batch_size=32, validation_data=(val_padded, val_y_onehot), callbacks=[earlystop_callback])


Epoch 1/100
85/85 [==============================] - 22s 26ms/step - loss: 0.5384 - accuracy: 0.5941 - val_loss: 0.4463 - val_accuracy: 0.7068
Epoch 2/100
85/85 [==============================] - 1s 17ms/step - loss: 0.4128 - accuracy: 0.7252 - val_loss: 0.4050 - val_accuracy: 0.7113
Epoch 3/100
85/85 [==============================] - 1s 15ms/step - loss: 0.3759 - accuracy: 0.7481 - val_loss: 0.3854 - val_accuracy: 0.7338
Epoch 4/100
85/85 [==============================] - 1s 15ms/step - loss: 0.3343 - accuracy: 0.7852 - val_loss: 0.3856 - val_accuracy: 0.7211
Epoch 5/100
85/85 [==============================] - 1s 14ms/step - loss: 0.3131 - accuracy: 0.7959 - val_loss: 0.3772 - val_accuracy: 0.7316
Epoch 6/100
85/85 [==============================] - 1s 15ms/step - loss: 0.2880 - accuracy: 0.8200 - val_loss: 0.3807 - val_accuracy: 0.7436
Epoch 7/100
85/85 [==============================] - 2s 19ms/step - loss: 0.2599 - accuracy: 0.8419 - val_loss: 0.3859 - val_accuracy: 0.7444
Epoch

In [ ]:
prediction = model.predict(val_padded)

p = []
for i in range(len(prediction)):
    a = []
    for j in range(3):
        a.append(round(prediction[i][j]))
    p.append(a)

from sklearn.metrics import classification_report
target_names = list(label_encoder.classes_)

r = classification_report(val_y_onehot, p, target_names=target_names, output_dict=True)

42/42 [==============================] - 1s 5ms/step


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
r

{'Direct Violence': {'precision': 0.6792452830188679,
  'recall': 0.5510204081632653,
  'f1-score': 0.6084507042253521,
  'support': 196},
 'Non-Violence': {'precision': 0.7576499388004896,
  'recall': 0.8633193863319386,
  'f1-score': 0.8070404172099087,
  'support': 717},
 'Passive Violence': {'precision': 0.7835051546391752,
  'recall': 0.5467625899280576,
  'f1-score': 0.6440677966101694,
  'support': 417},
 'micro avg': {'precision': 0.7537490134175217,
  'recall': 0.7180451127819549,
  'f1-score': 0.7354639969195224,
  'support': 1330},
 'macro avg': {'precision': 0.7401334588195109,
  'recall': 0.6537007948077539,
  'f1-score': 0.6865196393484768,
  'support': 1330},
 'weighted avg': {'precision': 0.7542020534407408,
  'recall': 0.7180451127819549,
  'f1-score': 0.7266771341008377,
  'support': 1330},
 'samples avg': {'precision': 0.7127819548872181,
  'recall': 0.7180451127819549,
  'f1-score': 0.7145363408521304,
  'support': 1330}}

In [ ]:
# Convert the dictionary to a DataFrame
df = pd.DataFrame.from_dict(r, orient='index')

# Print the DataFrame
print(df)

                  precision    recall  f1-score  support
Direct Violence    0.679245  0.551020  0.608451      196
Non-Violence       0.757650  0.863319  0.807040      717
Passive Violence   0.783505  0.546763  0.644068      417
micro avg          0.753749  0.718045  0.735464     1330
macro avg          0.740133  0.653701  0.686520     1330
weighted avg       0.754202  0.718045  0.726677     1330
samples avg        0.712782  0.718045  0.714536     1330
